# Rubik's Cube SAT Solver trial

## Modélisation: 

Le problème est un modèle dynamique, comme le cube prend différents états au cours du temps, suivant les actions qui sont choisies.
Il est donc nécessaire de prendre en compte la dimension temporelle dans notre problème, comme nous ne cherchons pas à déterminer un état final (car on connait l'état final), mais plutôt à déterminer les actions à effectuer pour passer d'un état initial à un état final.

Il faut donc modéliser d'une part les états du cube  et d'autre part les actions choisies à chaques étapes.

### Modélisation du cube : 

Pour chaque étapes, on modélise le cube en numérotant ses faces de 1 à 6, et en numérotant les cases de 1 à 4 pour chaques faces. Il nous faut aussi encoder la couleur de chaque case pour cela plusieurs choix s'offrent à nous:

![Rubik's Cube faces](./images/rubikscube1.png)
![Rubik's Cube facettes](./images/rubikscube2.png)

- En utilisant 6 variables pour chaques faces, chacune encodant la couleur. Alors seulement une variable parmis les 6 pourrat être vraie, et les autres fausses (c'est une contrainte). Cela donne $6*4*6=144$ variables pour chaques étapes.

- ou en encodant la couleur sur 3 bits, ce qui donne 8 couleurs possibles, et en interdisant 2 états pour réduire le nombre à 6 couleurs possible. Cela donne $6*4*3=72$ variables pour chaques étapes.

Pour commencer nous utiliserons la première méthode à 6 couleurs pour simplifier l'implémentation, et nous pourrons essayer la deuxième méthode si le temps de calcul est trop long.

### Modélisation des actions :

Pour modéliser les action du cubes il faut en premier lieux dénombrer le nombre d'actions possibles. Il y a en tout 18 mouvements posssible, pour chaque face on peut faire une rotation de 90° dans le sens horaire ou anti-horaire, ou une rotation de 180°, ce qui donne $6*3=18$ actions possibles.

En réalité beaucoup sont redondantes. Par exemple, si on fait une rotation de 90° dans le sens horaire de la face 1, cela revient à faire une rotation de 270° dans le sens anti-horaire de la face 1, ou une rotation de 90° dans le sens anti-horaire de la face 6. On peut réduire le nombre d'actions possible à différents niveau :

- 3 actions : rotation de 90° dans le sens horaire, pour la face du haut, du devant et de gauche (les faces 1,2,3). avec ces 3 actions on peut réaliser toutes les actions, c'est la modélisation minimale. Cependant cela ne nous permet pas de trouver le nombre d'actions minimales pour résoudre le cube, car il faudra plusieur actions répétés pour réaliser d'autres actions.

- 6 actions : pareil en rajoutant les rotations de 90° dans le sens antihoraire pour chaque face

- 9 actions : la manière optimale, en rajoutant les rotations de 180° pour les 3 faces. Avec ces 9 actions on peut réaliser toutes les actions possibles en 1 coup.

Nous utiliserons donc 9 actions pour modéliser les actions du cube, soit les rotations de 90° dans les 2 sens et les rotations de 180° pour chaque face.

Pour encoder ces 9 actions dans des variables propositionnelles, il existe plusieur manières:

- avec 9 variables propositionnelles, une pour chaque action

- avec 6 variables propositionnelle, 3 pour le choix de la face à tourner et 3 pour le choix du mouvement à réaliser (90° horaire, 90° anti-horaire, 180°)

- avec 4 variables en encodant le choix de la face à tourner sur 2 bits et le choix du mouvement sur 2 bits. ou en encodant directement le choix du mouvement sur 4 bits. Cette méthode est la plus compacte, mais elle est bien plus difficile à implémenter.

Nous utiliserons la première ou la 2eme méthode selon ce qui semble le plus simple à implémenter dans un premier temps.

### Modélisation de la résolution du cube : 

Il est important de remarquer que 

2 options s'offrent encore à nous:

- Proposition 1: on fixe le nombre d'étape, en commenceant par 1 étape. On implémente les contraintes CNF pour que le problème soit satisfait à la dernière étape. On résout le problème grâce au solveur, et si le problème est insatisfiable on recommence en incrémentant le nombre d'étape de 1, jusqu'à résolution du problème. Cette méthode nous donnera alors la solutions la plus courte possible.

- Proposition 2 : on fixe le nombre d'étape à 11, comme on sait que le cube est résolvable en 11 étapes maximum (litérature). Et on ajoute 11 variables au problème une variable i était vrai si à l'étape i le cube est résolu. Il faudra alors au moins 1 des variables i vraie pour que le problème soit satisfait. Cette modélisation ne donnera pas forcément la solution la plus courte, sauf peut être en ajoutant des contraintes supplémentaires (je ne vois pas encore lesquelles).

Pour avoir la solution optimale, nous allons nous concentrer sur le choix 1 pour l'instant.

Pour la contrainte de résolution, on force les 6 faces à avoir une unique couleur pour chacune des 6 cases. On peut aussi ajouter des contraintes pour forcer les couleurs des cases à être celles de l'état final, mais ça surcontraindrait le problème car la configuration finale est possible pour 24 état différents selons comment on tourne le cube, hors on cherche juste à résoudre le cube et pas un état final précis.


## Implémentation:


### Les variables propositionnelles:

In [77]:
# implémentation de la modélisation du probmème

import numpy as np 

t_max = 11 # (pour l'instant)

# le cube
n_faces = 6
n_cases = 4
n__couleurs = 6
R = np.arange(1, t_max*n_faces*n_cases*n__couleurs + 1).reshape(t_max, n_faces, n_cases, n__couleurs)

# les actions (3 mouvements et 3 faces)
n_mouvements = 3
n_faces = 3
M = np.arange(1, t_max*n_mouvements  + 1).reshape(t_max, n_mouvements)
M += np.max(R)
F = np.arange(1, t_max*n_faces  + 1).reshape(t_max, n_faces)
F += np.max(M)

print("on s'assure que 2 variables n'est pas le même nombre: ")
print("R :",R.min(),"->", R.max())
print("A :",M.min(),"->", M.max())
print("F :",F.min(),"->", F.max())
print("on a donc", F.max(), "variables propositionnelles pour notre modélisation.")



on s'assure que 2 variables n'est pas le même nombre: 
R : 1 -> 1584
A : 1585 -> 1617
F : 1618 -> 1650
on a donc 1650 variables propositionnelles pour notre modélisation.


### Contraintes d'unicité des mouvements:

In [68]:
# CONTRAINTES DE BASE POUR LE CUBE

# en général on utilisera 
# t pour la numéros de l'étape
# f pour la face
# i pour le numéro de la case
# c pour la couleur

basic_R_constraints = []

# chaque sommet doit être colorié d'au moins une couleur
for t,f,i in np.ndindex((t_max, n_faces, n_cases)):
    basic_R_constraints.append([int(R[t,f,i,c]) for c in range(6)])

# chaque sommet doit être colorié d'au plus une couleur
for t,f,i in np.ndindex((t_max,n_faces,n_cases)):
    for k in range(6):
        for l in range(k+1,6):
            basic_R_constraints.append([-int(R[t,f,i,k]), -int(R[t,f,i,l])])
            
            
print("nombre de contraintes:", len(basic_R_constraints))


nombre de contraintes: 2112


### Contraintes d'unicité des mouvements:

In [72]:
# CONTRAINTES DE BASE POUR LES MOUVEMENTS

basic_moves_constraints = []

# il doit y avoir au moins un mouvement sur une face à chaque étape,
for t in range(t_max):
    basic_moves_constraints.append([int(M[t,k]) for k in range(3)])
    basic_moves_constraints.append([int(F[t,k]) for k in range(3)])
    
    
# il doit y avoir au plus un mouvement sur une face à chaque étape
for t in range(t_max):
    for k in range(3):
        for l in range(k+1,3):
            basic_moves_constraints.append([-int(M[t,k]), -int(M[t,l])])
            basic_moves_constraints.append([-int(F[t,k]), -int(F[t,l])])
            
print("nombre de contraintes:", len(basic_moves_constraints))

nombre de contraintes: 88


### Contraintes de résolution du cube:

In [76]:
# Contraite de résolution du problème

# à la dernière étape, pour chaque face, 
# les 3 premières facettes doivent être de la même couleur que la 4ème

# on peut mettre la contrainte sous forme d'égalité, mais cela nous rajoutera 2 contrainte par égalité,
# à la place ou peut simplement la mettre sous forme d'implication: 
# "pour 2 facettes A et B, pour toutes couleurs k, si A[k] alors B[k]"
# comme chaque case est déjà contrainte à n'avoir qu'une seule et même couleur, cela revient au même.
# on utilisera donc toujours l'implication au lieu de l'égalité, ce qui divise le nombre de contraintes par 2.

# on peut le prouver asser facilement:
# pour k such as Ak = 1,  Ak => Bk , then Bk = 1 , and then for all f !=k , Af=0 & Bf=0,

faisability_constraints= []

def equal_to(a,b):
    return [[-a,b],[a,-b]]

def implies(a,b):
    return [[-a,b]]

t = t_max-1

for f in range(6):
    for c in range(3):
        for k in range(6):
            faisability_constraints += implies(R[t,f,c,k], R[t,f,3,k])
            
print( "nombre de contraintes:", len(faisability_constraints))

nombre de contraintes: 108


### Contraintes de mouvement

Pour appliquer les contraintes de mouvements (cf de rotation), il va faloir définir chacun des mouvements pour chacunes des faces, pour nous simplifier le travail, on remodélise les 3 faces de la même façon pour n'avoir à définir les 3 mouvements différents que pour une seule face.

on renome les facettes de la façon suivante pour chaque face (par exemple pour la face 2):

![Rubik's Cube facettes](./images/faces_renum.png)

On rappelle la modélisation du cube : 

![Rubik's Cube faces](./images/rubikscube1.png)
![Rubik's Cube facettes](./images/rubikscube2.png)

In [92]:
f = lambda i: (i-2) % 8

for i in range(0,8):
    print(i, f(i))   

0 6
1 7
2 0
3 1
4 2
5 3
6 4
7 5


In [ ]:
# rotation horaire face 1

# on cherche à traduire des implications découlant des actions choisies

# exemple sens horaire : 
# F(t,1) \and M(t,1) => R(t,1,1,:) = R(t+1,1,2,:)
# on peut déjà le transformer en une implication comme expliqué plus haut:
# F(t,1) \and M(t,1) => ( R(t,1,1,:) => R(t+1,1,2,:) ) 
# on peut la transformer en forme normale conjonctive:
# ( -F(t,1) \or -M(t,1) \or -R(t,1,1,:) \or R(t+1,1,2,:) )

# les mouvements sont numérotés comme suis:
# M[t,1] : sens horaire
# M[t,2] : sens anti-horaire
# M[t,3] : sens horaire



adjacences = {
    1 : ( (4,1), (4,2), (3,1), (3,2), (2,1), (2,2), (5,1), (5,2) ),
    2 : ( (1,4), (1,3), (3,2), (3,4), (6,1), (6,2), (5,3), (5,1) ),
    3 : ( (1,3), (1,1), (4,2), (4,4), (6,3), (6,1), (2,3), (2,1) ),
}

def rotate_90(f, t):
    C = []
    # face principale
    implications = [(1,2), (2,4), (4,3), (3,1)]
    for c1, c2 in implications:
        for k in range(6):
            C.append([-F[t,f], -M[t,1], -R[t,f,c1,k], R[t+1,f,c2,k]])
    # facettes adjacentes:
    ordre = adjacences[f]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[ (i-2) % 8]
        for k in range(6):
            C.append([-F[t,f], -M[t,1], -R[t,f1,c1,k], R[t+1,f2,c2,k]])
    # les autres facettes (qui ne bougent pas):
    
        
def rotate_90_a(f, t):
    C = []
    # face principale
    implications = [(1,2), (2,4), (4,3), (3,1)]
    for c1, c2 in implications:
        for k in range(6):
            C.append([-F[t,f], -M[t,1], -R[t,f,c1,k], R[t+1,f,c2,k]])
    # facettes adjacentes:
    ordre = adjacences[f]
    for i in range(len(ordre)):

    
    





0 0 0
0 0 1
0 0 2
0 0 3
0 1 0
0 1 1
0 1 2
0 1 3
0 2 0
0 2 1
0 2 2
0 2 3
1 0 0
1 0 1
1 0 2
1 0 3
1 1 0
1 1 1
1 1 2
1 1 3
1 2 0
1 2 1
1 2 2
1 2 3
2 0 0
2 0 1
2 0 2
2 0 3
2 1 0
2 1 1
2 1 2
2 1 3
2 2 0
2 2 1
2 2 2
2 2 3
3 0 0
3 0 1
3 0 2
3 0 3
3 1 0
3 1 1
3 1 2
3 1 3
3 2 0
3 2 1
3 2 2
3 2 3
4 0 0
4 0 1
4 0 2
4 0 3
4 1 0
4 1 1
4 1 2
4 1 3
4 2 0
4 2 1
4 2 2
4 2 3
5 0 0
5 0 1
5 0 2
5 0 3
5 1 0
5 1 1
5 1 2
5 1 3
5 2 0
5 2 1
5 2 2
5 2 3
6 0 0
6 0 1
6 0 2
6 0 3
6 1 0
6 1 1
6 1 2
6 1 3
6 2 0
6 2 1
6 2 2
6 2 3
7 0 0
7 0 1
7 0 2
7 0 3
7 1 0
7 1 1
7 1 2
7 1 3
7 2 0
7 2 1
7 2 2
7 2 3
8 0 0
8 0 1
8 0 2
8 0 3
8 1 0
8 1 1
8 1 2
8 1 3
8 2 0
8 2 1
8 2 2
8 2 3
9 0 0
9 0 1
9 0 2
9 0 3
9 1 0
9 1 1
9 1 2
9 1 3
9 2 0
9 2 1
9 2 2
9 2 3
10 0 0
10 0 1
10 0 2
10 0 3
10 1 0
10 1 1
10 1 2
10 1 3
10 2 0
10 2 1
10 2 2
10 2 3
